In [1]:
from sympy import init_printing
init_printing()

In [2]:
from sympy import Symbol,Matrix,symbols
from sympy import I,conjugate,re,im
from sympy import sin,cos,exp,sqrt,pi
from sympy import pprint
from sympy import simplify
from sympy import KroneckerDelta
from sympy import Function, Derivative
from sympy import solve
from sympy import zeros
from sympy import Integer

from fast.symbolic import define_density_matrix, define_laser_variables, polarization_vector
from fast.symbolic import cartesian_to_helicity, helicity_to_cartesian, helicity_dot_product
from fast.symbolic import define_r_components, define_frequencies
from fast.symbolic import delta_greater, delta_lesser
from fast.symbolic import ket,bra,lindblad_operator,lindblad_terms
from fast.symbolic import define_psi_coefficients
from fast.symbolic import define_rho_vector,calculate_A_b


We define the number of states and of radiation fields.

In [3]:
Ne=2
Nl=1

We define the variables related to the laser field.

In [4]:
E0,omega_laser=define_laser_variables(Nl)
pprint(E0)
pprint(omega_laser)

[E¹₀]
[ω¹]


We define a few important symbols.

In [5]:
t,hbar,e=symbols("t hbar e",positive=True)
pprint([t,hbar,e])

[t, h̅, e]


We write an electric field propagating trough the $\hat{x}$ direction polarized in the $\hat{z}$ direction. First the wave vector:

In [6]:
phi=0; theta=pi/2; alpha=pi/2; beta=0

k=Matrix([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])
pprint(k)

⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦


The polarization vectors.

In [7]:
ep=polarization_vector(phi,theta,alpha,beta, 1)
em=polarization_vector(phi,theta,alpha,beta,-1)
pprint([ep,em])

⎡⎡0⎤, ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎣⎣1⎦  ⎣1⎦⎦


The electric field (evaluated in $\vec{R}=0$).

In [8]:
E_cartesian=(E0[0]/2*ep*exp(-I*omega_laser[0]*t) + E0[0].conjugate()/2*em*exp( I*omega_laser[0]*t))
pprint(E_cartesian)

⎡            0             ⎤
⎢                          ⎥
⎢            0             ⎥
⎢                          ⎥
⎢     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎥
⎢E¹₀⋅ℯ          ℯ      ⋅E¹₀⎥
⎢──────────── + ───────────⎥
⎣     2              2     ⎦


We write the electric field in the helicity basis.

In [9]:
E=cartesian_to_helicity(E_cartesian)
pprint(E)

⎡            0             ⎤
⎢                          ⎥
⎢     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎥
⎢E¹₀⋅ℯ          ℯ      ⋅E¹₀⎥
⎢──────────── + ───────────⎥
⎢     2              2     ⎥
⎢                          ⎥
⎣            0             ⎦


We define the position operator.

In [10]:
r=define_r_components(Ne,helicity=True,explicitly_hermitian=True)
pprint(r)

⎡⎡    0      -r_{+1;21}⎤, ⎡   0      r_{0;21}⎤, ⎡    0      -r_{-1;21}⎤⎤
⎢⎢                     ⎥  ⎢                  ⎥  ⎢                     ⎥⎥
⎣⎣r_{-1;21}      0     ⎦  ⎣r_{0;21}     0    ⎦  ⎣r_{+1;21}      0     ⎦⎦


The frequencies of the energy levels, the resonant frequencies, and the decay frequencies.

In [11]:
omega_level,omega,gamma=define_frequencies(Ne,explicitly_antisymmetric=True)
pprint(omega_level)
print
pprint(omega)
print
pprint(gamma)

[ω₁, ω₂]

⎡ 0   -ω₂₁⎤
⎢         ⎥
⎣ω₂₁   0  ⎦

⎡ 0   -γ₂₁⎤
⎢         ⎥
⎣γ₂₁   0  ⎦


The atomic hamiltonian is

In [12]:
H0=Matrix([[hbar*omega_level[i]*KroneckerDelta(i,j) for j in range(Ne)] for i in range(Ne)])
pprint(H0)

⎡h̅⋅ω₁    0  ⎤
⎢            ⎥
⎣  0    h̅⋅ω₂⎦


The interaction hamiltonian is

In [13]:
H1=e*helicity_dot_product(E,r)
pprint(H1,num_columns=120)

⎡                                                    ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞⎤
⎢                                                    ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟⎥
⎢                   0                     e⋅r_{0;21}⋅⎜──────────── + ───────────⎟⎥
⎢                                                    ⎝     2              2     ⎠⎥
⎢                                                                                ⎥
⎢           ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞                                         ⎥
⎢           ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟                                         ⎥
⎢e⋅r_{0;21}⋅⎜──────────── + ───────────⎟                     0                   ⎥
⎣           ⎝     2              2     ⎠                                         ⎦


and the complete hamiltonian is

In [14]:
H=H0+H1
pprint(H,num_columns=120)

⎡                                                    ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞⎤
⎢                                                    ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟⎥
⎢                 h̅⋅ω₁                   e⋅r_{0;21}⋅⎜──────────── + ───────────⎟⎥
⎢                                                    ⎝     2              2     ⎠⎥
⎢                                                                                ⎥
⎢           ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞                                         ⎥
⎢           ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟                                         ⎥
⎢e⋅r_{0;21}⋅⎜──────────── + ───────────⎟                   h̅⋅ω₂                 ⎥
⎣           ⎝     2              2     ⎠                                         ⎦


# Rotating wave approximation
Notice that the electric field can be separated by terms with positive and negative frequency:

In [15]:
E_cartesian_p=E0[0]            /2*ep*exp(-I*omega_laser[0]*t)
E_cartesian_m=E0[0].conjugate()/2*em*exp( I*omega_laser[0]*t)

E_p=cartesian_to_helicity(E_cartesian_p)
E_m=cartesian_to_helicity(E_cartesian_m)

pprint([E_p,E_m])

⎡⎡     0      ⎤, ⎡     0     ⎤⎤
⎢⎢            ⎥  ⎢           ⎥⎥
⎢⎢     -ⅈ⋅ω¹⋅t⎥  ⎢ ⅈ⋅ω¹⋅t ___⎥⎥
⎢⎢E¹₀⋅ℯ       ⎥  ⎢ℯ      ⋅E¹₀⎥⎥
⎢⎢────────────⎥  ⎢───────────⎥⎥
⎢⎢     2      ⎥  ⎢     2     ⎥⎥
⎢⎢            ⎥  ⎢           ⎥⎥
⎣⎣     0      ⎦  ⎣     0     ⎦⎦


In [16]:
pprint( simplify(E-(E_p+E_m)) )

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦


The position operator can also be separated in this way. We go to the interaction picture (with $\hat{H}_0$ as the undisturbed hamiltonian)

In [17]:
r_I=[ Matrix([[exp(I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)] for i in range(Ne)]) for p in range(3)]
pprint(r_I[0])
pprint(r_I[1])
pprint(r_I[2])

⎡                                -ⅈ⋅ω₂₁⋅t⎤
⎢        0           -r_{+1;21}⋅ℯ        ⎥
⎢                                        ⎥
⎢           ⅈ⋅ω₂₁⋅t                      ⎥
⎣r_{-1;21}⋅ℯ                  0          ⎦
⎡                             -ⅈ⋅ω₂₁⋅t⎤
⎢        0          r_{0;21}⋅ℯ        ⎥
⎢                                     ⎥
⎢          ⅈ⋅ω₂₁⋅t                    ⎥
⎣r_{0;21}⋅ℯ                 0         ⎦
⎡                                -ⅈ⋅ω₂₁⋅t⎤
⎢        0           -r_{-1;21}⋅ℯ        ⎥
⎢                                        ⎥
⎢           ⅈ⋅ω₂₁⋅t                      ⎥
⎣r_{+1;21}⋅ℯ                  0          ⎦


Which can be decomposed as

In [18]:
r_I_p=[ Matrix([[ delta_greater(j,i)*exp(-I*omega[j,i]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_p[0])
pprint(r_I_p[1])
pprint(r_I_p[2])

⎡               -ⅈ⋅ω₂₁⋅t⎤
⎢0  -r_{+1;21}⋅ℯ        ⎥
⎢                       ⎥
⎣0           0          ⎦
⎡             -ⅈ⋅ω₂₁⋅t⎤
⎢0  r_{0;21}⋅ℯ        ⎥
⎢                     ⎥
⎣0          0         ⎦
⎡               -ⅈ⋅ω₂₁⋅t⎤
⎢0  -r_{-1;21}⋅ℯ        ⎥
⎢                       ⎥
⎣0           0          ⎦


In [19]:
r_I_m=[ Matrix([[ delta_lesser( j,i)*exp( I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_m[0])
pprint(r_I_m[1])
pprint(r_I_m[2])

⎡        0           0⎤
⎢                     ⎥
⎢           ⅈ⋅ω₂₁⋅t   ⎥
⎣r_{-1;21}⋅ℯ         0⎦
⎡        0          0⎤
⎢                    ⎥
⎢          ⅈ⋅ω₂₁⋅t   ⎥
⎣r_{0;21}⋅ℯ         0⎦
⎡        0           0⎤
⎢                     ⎥
⎢           ⅈ⋅ω₂₁⋅t   ⎥
⎣r_{+1;21}⋅ℯ         0⎦


that summed equal $\vec{\hat{r}}_I$

In [20]:
pprint( [r_I[p]-(r_I_p[p]+r_I_m[p]) for p in range(3)] )

⎡⎡0  0⎤, ⎡0  0⎤, ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦  ⎣0  0⎦⎦


Thus the interaction hamiltonian in the interaciton picture is
\begin{equation}
    \hat{H}_{1I}=e\vec{E}\cdot \vec{\hat{r}}_I= e(\vec{E}^{(+)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(-)}_I)
\end{equation}

In [21]:
H1I=e*helicity_dot_product(E,r_I)
pprint(H1I,num_columns=120)

⎡                                                             ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞          ⎤
⎢                                                             ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟  -ⅈ⋅ω₂₁⋅t⎥
⎢                       0                          e⋅r_{0;21}⋅⎜──────────── + ───────────⎟⋅ℯ        ⎥
⎢                                                             ⎝     2              2     ⎠          ⎥
⎢                                                                                                   ⎥
⎢           ⎛     -ⅈ⋅ω¹⋅t    ⅈ⋅ω¹⋅t ___⎞                                                            ⎥
⎢           ⎜E¹₀⋅ℯ          ℯ      ⋅E¹₀⎟  ⅈ⋅ω₂₁⋅t                                                   ⎥
⎢e⋅r_{0;21}⋅⎜──────────── + ───────────⎟⋅ℯ                                 0                        ⎥
⎣           ⎝     2              2     ⎠                                                            ⎦


Since both $\omega^l$ and $\omega_{ij}$ are in the order of THz, the terms that have frequencies with the same sign are summed, and thus also of the order of THz. The frequencies in the terms with oposite signs however, are detunings of the order of MHz. Since we are only interested in the coarse-grained evolution of the density matrix, we may omit the fast terms and approximate

\begin{equation}
    \hat{H}_{1I} \simeq \hat{H}_{1I,RWA}= e( \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I )
\end{equation}

That is known as the rotating wave approximation (RWA).

In [22]:
H1IRWA=e*(helicity_dot_product(E_p,r_I_m)+helicity_dot_product(E_m,r_I_p))
pprint(H1IRWA)

⎡                                              ⅈ⋅ω¹⋅t  -ⅈ⋅ω₂₁⋅t ___⎤
⎢                                  e⋅r_{0;21}⋅ℯ      ⋅ℯ        ⋅E¹₀⎥
⎢               0                  ────────────────────────────────⎥
⎢                                                 2                ⎥
⎢                                                                  ⎥
⎢                -ⅈ⋅ω¹⋅t  ⅈ⋅ω₂₁⋅t                                  ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅ℯ       ⋅ℯ                                         ⎥
⎢────────────────────────────────                 0                ⎥
⎣               2                                                  ⎦


 Returning to the Schrödinger picture we have.

In [23]:
r_p=[ Matrix([[ delta_greater(j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_p)

⎡⎡0  -r_{+1;21}⎤, ⎡0  r_{0;21}⎤, ⎡0  -r_{-1;21}⎤⎤
⎢⎢             ⎥  ⎢           ⎥  ⎢             ⎥⎥
⎣⎣0      0     ⎦  ⎣0     0    ⎦  ⎣0      0     ⎦⎦


In [24]:
r_m=[ Matrix([[ delta_lesser( j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_m)

⎡⎡    0      0⎤, ⎡   0      0⎤, ⎡    0      0⎤⎤
⎢⎢            ⎥  ⎢           ⎥  ⎢            ⎥⎥
⎣⎣r_{-1;21}  0⎦  ⎣r_{0;21}  0⎦  ⎣r_{+1;21}  0⎦⎦


In [25]:
pprint( [r[p]-(r_p[p]+r_m[p]) for p in range(3)] )

⎡⎡0  0⎤, ⎡0  0⎤, ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦  ⎣0  0⎦⎦


Thus the interaction hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [26]:
H1RWA=e*(helicity_dot_product(E_p,r_m)+helicity_dot_product(E_m,r_p))
pprint(H1RWA)

⎡                                     ⅈ⋅ω¹⋅t ___⎤
⎢                         e⋅r_{0;21}⋅ℯ      ⋅E¹₀⎥
⎢           0             ──────────────────────⎥
⎢                                   2           ⎥
⎢                                               ⎥
⎢                -ⅈ⋅ω¹⋅t                        ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅ℯ                               ⎥
⎢───────────────────────            0           ⎥
⎣           2                                   ⎦


And the complete hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [27]:
HRWA=H0+H1RWA
pprint(HRWA)

⎡                                     ⅈ⋅ω¹⋅t ___⎤
⎢                         e⋅r_{0;21}⋅ℯ      ⋅E¹₀⎥
⎢         h̅⋅ω₁           ──────────────────────⎥
⎢                                   2           ⎥
⎢                                               ⎥
⎢                -ⅈ⋅ω¹⋅t                        ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅ℯ                               ⎥
⎢───────────────────────          h̅⋅ω₂         ⎥
⎣           2                                   ⎦


# Rotating Frame
Next we will make a phase transformation in order to eliminate the explicit time dependance of the equations.

In [28]:
c,ctilde,phase=define_psi_coefficients(Ne)
pprint([c,ctilde,phase])

⎡⎡c₁(t)⎤, ⎡\tilde{c}_{1}(t)⎤, ⎡θ₁⎤⎤
⎢⎢     ⎥  ⎢                ⎥  ⎢  ⎥⎥
⎣⎣c₂(t)⎦  ⎣\tilde{c}_{2}(t)⎦  ⎣θ₂⎦⎦


In [29]:
psi=Matrix([ exp(I*phase[i]*t)*ctilde[i] for i in range(Ne)])
pprint(psi)

⎡                  ⅈ⋅t⋅θ₁⎤
⎢\tilde{c}_{1}(t)⋅ℯ      ⎥
⎢                        ⎥
⎢                  ⅈ⋅t⋅θ₂⎥
⎣\tilde{c}_{2}(t)⋅ℯ      ⎦


The Schrödinger equation $i\hbar \partial_t |\psi\rangle=\hat{H}_{RWA}$ is

In [30]:
lhs=Matrix([(I*hbar*Derivative(psi[i],t).doit()).expand() for i in range(Ne)])
pprint(lhs)

⎡                         ⅈ⋅t⋅θ₁⎤
⎢-h̅⋅θ₁⋅\tilde{c}_{1}(t)⋅ℯ      ⎥
⎢                               ⎥
⎢                         ⅈ⋅t⋅θ₂⎥
⎣-h̅⋅θ₂⋅\tilde{c}_{2}(t)⋅ℯ      ⎦


In [31]:
rhs=HRWA*psi
pprint(rhs,num_columns=120)

⎡                             ⅈ⋅ω¹⋅t  ⅈ⋅t⋅θ₂ ___                                  ⎤
⎢e⋅r_{0;21}⋅\tilde{c}_{2}(t)⋅ℯ      ⋅ℯ      ⋅E¹₀                           ⅈ⋅t⋅θ₁ ⎥
⎢─────────────────────────────────────────────── + h̅⋅ω₁⋅\tilde{c}_{1}(t)⋅ℯ       ⎥
⎢                       2                                                         ⎥
⎢                                                                                 ⎥
⎢                                 -ⅈ⋅ω¹⋅t  ⅈ⋅t⋅θ₁                                 ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅\tilde{c}_{1}(t)⋅ℯ       ⋅ℯ                                 ⅈ⋅t⋅θ₂⎥
⎢──────────────────────────────────────────────── + h̅⋅ω₂⋅\tilde{c}_{2}(t)⋅ℯ      ⎥
⎣                       2                                                         ⎦


We multiply each of these equations by $e^{-i \theta_i t}$ and substracting $i \theta_i \tilde{c}_i$

In [32]:
lhs_new=Matrix([simplify(  lhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(lhs_new)

⎡0⎤
⎢ ⎥
⎣0⎦


In [33]:
rhs_new=Matrix([simplify(  rhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(rhs_new,num_columns=120)

⎡                             ⅈ⋅ω¹⋅t  -ⅈ⋅t⋅θ₁  ⅈ⋅t⋅θ₂ ___                                                   ⎤
⎢e⋅r_{0;21}⋅\tilde{c}_{2}(t)⋅ℯ      ⋅ℯ       ⋅ℯ      ⋅E¹₀                                                   ⎥
⎢──────────────────────────────────────────────────────── + h̅⋅ω₁⋅\tilde{c}_{1}(t) + h̅⋅θ₁⋅\tilde{c}_{1}(t) ⎥
⎢                           2                                                                               ⎥
⎢                                                                                                           ⎥
⎢                                 -ⅈ⋅ω¹⋅t  ⅈ⋅t⋅θ₁  -ⅈ⋅t⋅θ₂                                                  ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅\tilde{c}_{1}(t)⋅ℯ       ⋅ℯ      ⋅ℯ                                                         ⎥
⎢───────────────────────────────────────────────────────── + h̅⋅ω₂⋅\tilde{c}_{2}(t) + h̅⋅θ₂⋅\tilde{c}_{2}(t)⎥
⎣                            2                                                                              ⎦


It can be seen that the equations loose their explicit time dependance only if $\omega^{1} - \theta_{1} + \theta_{2}=0$. Which is satisfied if

In [34]:
phase_transformation=solve(omega_laser[0]+phase[1]-phase[0],phase[1],dict=True)[0]
pprint(phase_transformation)

{θ₂: -ω¹ + θ₁}


There is a free parameter $\theta_1$, which is to be expected, since state vetors $|\psi\rangle$ always have a global phase invariance

In [35]:
pprint(psi.subs(phase_transformation))

⎡                      ⅈ⋅t⋅θ₁    ⎤
⎢    \tilde{c}_{1}(t)⋅ℯ          ⎥
⎢                                ⎥
⎢                  ⅈ⋅t⋅(-ω¹ + θ₁)⎥
⎣\tilde{c}_{2}(t)⋅ℯ              ⎦


Thus the equations become

In [36]:
pprint(lhs_new)

⎡0⎤
⎢ ⎥
⎣0⎦


In [37]:
rhs_new=simplify(rhs_new.subs(phase_transformation))
pprint(rhs_new)

⎡                               ___                                   ⎤
⎢   e⋅r_{0;21}⋅\tilde{c}_{2}(t)⋅E¹₀                                   ⎥
⎢   ─────────────────────────────── + h̅⋅(ω₁ + θ₁)⋅\tilde{c}_{1}(t)   ⎥
⎢                  2                                                  ⎥
⎢                                                                     ⎥
⎢E¹₀⋅e⋅r_{0;21}⋅\tilde{c}_{1}(t)                                      ⎥
⎢─────────────────────────────── + h̅⋅(-ω¹ + ω₂ + θ₁)⋅\tilde{c}_{2}(t)⎥
⎣               2                                                     ⎦


It can be seen that this is the Schrödinger equation derived from an effective hamiltonian $\tilde{H}$

In [38]:
Htilde=Matrix([ [Derivative(rhs_new[i],ctilde[j]).doit() for j in range(Ne)] for i in range(Ne)])
pprint(Htilde)

⎡                             ___  ⎤
⎢                  e⋅r_{0;21}⋅E¹₀  ⎥
⎢ h̅⋅(ω₁ + θ₁)     ──────────────  ⎥
⎢                        2         ⎥
⎢                                  ⎥
⎢E¹₀⋅e⋅r_{0;21}                    ⎥
⎢──────────────  h̅⋅(-ω¹ + ω₂ + θ₁)⎥
⎣      2                           ⎦


We can see that it is convenient to choose $\theta_1=-\omega_1$ to simplify the hamiltonian. Also, we can recognize $\omega^1-\omega_2+\omega_1=\delta$ as the detuning of the laser field relative to the atomic transition $\omega_{21}=\omega_2-\omega_1$.

In [39]:
delta=Symbol("delta",real=True)
Htilde=Htilde.subs({phase[0]:-omega_level[0]}).subs({omega_laser[0]:delta+omega_level[1]-omega_level[0]})
pprint(Htilde)

⎡                           ___⎤
⎢                e⋅r_{0;21}⋅E¹₀⎥
⎢      0         ──────────────⎥
⎢                      2       ⎥
⎢                              ⎥
⎢E¹₀⋅e⋅r_{0;21}                ⎥
⎢──────────────      -δ⋅h̅     ⎥
⎣      2                       ⎦


If we define the Rabi frequency $\Omega =e E_0^1 r_{0;21}/\hbar$

In [40]:
Omega=Symbol("Omega",real=False)
Htilde=Htilde.subs({E0[0]:Omega*hbar/r[1][1,0]/e})
pprint(Htilde)

⎡         _ ⎤
⎢      h̅⋅Ω ⎥
⎢ 0    ──── ⎥
⎢       2   ⎥
⎢           ⎥
⎢Ω⋅h̅       ⎥
⎢────  -δ⋅h̅⎥
⎣ 2         ⎦


We define the density matrix.

In [41]:
rho=define_density_matrix(Ne)
pprint( rho )

⎡ρ₁₁  ρ₁₂⎤
⎢        ⎥
⎣ρ₂₁  ρ₂₂⎦


The hamiltonian part of the equations is
\begin{equation}
    \dot{\hat{\rho}}=\frac{i}{\hbar}[\hat{\rho}, \hat{\tilde{H}}]
\end{equation}

In [42]:
hamiltonian_terms=(I/hbar*(rho*Htilde-Htilde*rho)).expand()
pprint(hamiltonian_terms)

⎡                      _                         _         _⎤
⎢      ⅈ⋅Ω⋅ρ₁₂   ⅈ⋅ρ₂₁⋅Ω                   ⅈ⋅ρ₁₁⋅Ω   ⅈ⋅ρ₂₂⋅Ω⎥
⎢      ─────── - ───────        -ⅈ⋅δ⋅ρ₁₂ + ─────── - ───────⎥
⎢         2         2                         2         2   ⎥
⎢                                                           ⎥
⎢                                                     _     ⎥
⎢  ⅈ⋅Ω⋅ρ₁₁   ⅈ⋅Ω⋅ρ₂₂                  ⅈ⋅Ω⋅ρ₁₂   ⅈ⋅ρ₂₁⋅Ω     ⎥
⎢- ─────── + ─────── + ⅈ⋅δ⋅ρ₂₁      - ─────── + ───────     ⎥
⎣     2         2                        2         2        ⎦


There is only one Lindblad operator, since there is only one spontaneous decay channel.

In [43]:
lindblad_terms=gamma[1,0]*lindblad_operator(ket(1,Ne)*bra(2,Ne),rho)
pprint(lindblad_terms, num_columns=120)

⎡           -γ₂₁⋅ρ₁₂ ⎤
⎢ γ₂₁⋅ρ₂₂   ─────────⎥
⎢               2    ⎥
⎢                    ⎥
⎢-γ₂₁⋅ρ₂₁            ⎥
⎢─────────  -γ₂₁⋅ρ₂₂ ⎥
⎣    2               ⎦


# Optical Bloch Equations
The Optical Bloch equations are thus.

In [44]:
eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120)

⎡                                _                                   _         _⎤
⎢      ⅈ⋅Ω⋅ρ₁₂             ⅈ⋅ρ₂₁⋅Ω                   γ₂₁⋅ρ₁₂   ⅈ⋅ρ₁₁⋅Ω   ⅈ⋅ρ₂₂⋅Ω⎥
⎢      ─────── + γ₂₁⋅ρ₂₂ - ───────        -ⅈ⋅δ⋅ρ₁₂ - ─────── + ─────── - ───────⎥
⎢         2                   2                         2         2         2   ⎥
⎢                                                                               ⎥
⎢                                                                         _     ⎥
⎢  ⅈ⋅Ω⋅ρ₁₁   ⅈ⋅Ω⋅ρ₂₂             γ₂₁⋅ρ₂₁        ⅈ⋅Ω⋅ρ₁₂             ⅈ⋅ρ₂₁⋅Ω     ⎥
⎢- ─────── + ─────── + ⅈ⋅δ⋅ρ₂₁ - ───────      - ─────── - γ₂₁⋅ρ₂₂ + ───────     ⎥
⎣     2         2                   2              2                   2        ⎦


which is how most literature will show the equations. However, a more convenient way to express this equations is to explicitly asume a normalized and hermitian density matrix

In [45]:
rho=define_density_matrix(Ne,explicitly_hermitian=True,normalized=True)
pprint( rho )

⎡          ___⎤
⎢-ρ₂₂ + 1  ρ₂₁⎥
⎢             ⎥
⎣  ρ₂₁     ρ₂₂⎦


In [46]:
hamiltonian_terms = (I/hbar*(rho*Htilde-Htilde*rho)).expand()
lindblad_terms    =gamma[1,0]*lindblad_operator(ket(1,Ne)*bra(2,Ne),rho)
eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120)

⎡       ___                   _                     ___               _⎤
⎢   ⅈ⋅Ω⋅ρ₂₁             ⅈ⋅ρ₂₁⋅Ω           ___   γ₂₁⋅ρ₂₁         _   ⅈ⋅Ω⎥
⎢   ─────── + γ₂₁⋅ρ₂₂ - ───────     - ⅈ⋅δ⋅ρ₂₁ - ─────── - ⅈ⋅ρ₂₂⋅Ω + ───⎥
⎢      2                   2                       2                 2 ⎥
⎢                                                                      ⎥
⎢                                            ___                   _   ⎥
⎢          ⅈ⋅Ω             γ₂₁⋅ρ₂₁       ⅈ⋅Ω⋅ρ₂₁             ⅈ⋅ρ₂₁⋅Ω   ⎥
⎢ⅈ⋅Ω⋅ρ₂₂ - ─── + ⅈ⋅δ⋅ρ₂₁ - ───────     - ─────── - γ₂₁⋅ρ₂₂ + ───────   ⎥
⎣           2                 2             2                   2      ⎦


and only consider the equations for the populations $\rho_{ii}$ for $i>1$ and the real and imaginary parts of the coherences below the diagonal.

In [47]:
ss_comp={ rho[i,j]:re(rho[i,j])+I*im(rho[i,j]) for j in range(Ne) for i in range(Ne)}
pprint( re(eqs[1,1].subs(ss_comp)) )

-γ₂₁⋅ρ₂₂ - re(Ω)⋅im(ρ₂₁) + re(ρ₂₁)⋅im(Ω)


In [48]:
pprint( re(eqs[1,0].subs(ss_comp)) )

             γ₂₁⋅re(ρ₂₁)               im(Ω)
-δ⋅im(ρ₂₁) - ─────────── - ρ₂₂⋅im(Ω) + ─────
                  2                      2  


In [49]:
pprint( im(eqs[1,0].subs(ss_comp)) )

            γ₂₁⋅im(ρ₂₁)               re(Ω)
δ⋅re(ρ₂₁) - ─────────── + ρ₂₂⋅re(Ω) - ─────
                 2                      2  


If the density matrix is represented as a vector whose components are the these independent components of the density matrix

In [50]:
rho_vect=define_rho_vector(rho,Ne)
pprint(rho_vect)

⎡  ρ₂₂  ⎤
⎢       ⎥
⎢re(ρ₂₁)⎥
⎢       ⎥
⎣im(ρ₂₁)⎦


Then the equations can be re-written as linear combinations of these components plus an independent term.
\begin{equation}
    \dot{\vec{\rho}} = \hat{A} \vec{\rho} + \vec{b}
\end{equation}
with $\hat{A}$ a linear operator acting in this vector space and $\vec{b}$ the vector of independent terms.

In [51]:
A,b=calculate_A_b(eqs,rho,Ne)
pprint([A,b])

⎡⎡ -γ₂₁   im(Ω)  -re(Ω)⎤, ⎡   0   ⎤⎤
⎢⎢                     ⎥  ⎢       ⎥⎥
⎢⎢        -γ₂₁         ⎥  ⎢-im(Ω) ⎥⎥
⎢⎢-im(Ω)  ─────    -δ  ⎥  ⎢───────⎥⎥
⎢⎢          2          ⎥  ⎢   2   ⎥⎥
⎢⎢                     ⎥  ⎢       ⎥⎥
⎢⎢               -γ₂₁  ⎥  ⎢ re(Ω) ⎥⎥
⎢⎢re(Ω)     δ    ───── ⎥  ⎢ ───── ⎥⎥
⎣⎣                 2   ⎦  ⎣   2   ⎦⎦


Explicitly, this is

In [52]:
eqs_new=A*rho_vect - b
pprint(eqs_new)

⎡  -γ₂₁⋅ρ₂₂ - re(Ω)⋅im(ρ₂₁) + re(ρ₂₁)⋅im(Ω)  ⎤
⎢                                            ⎥
⎢             γ₂₁⋅re(ρ₂₁)               im(Ω)⎥
⎢-δ⋅im(ρ₂₁) - ─────────── - ρ₂₂⋅im(Ω) + ─────⎥
⎢                  2                      2  ⎥
⎢                                            ⎥
⎢            γ₂₁⋅im(ρ₂₁)               re(Ω) ⎥
⎢δ⋅re(ρ₂₁) - ─────────── + ρ₂₂⋅re(Ω) - ───── ⎥
⎣                 2                      2   ⎦


Which is the same as the equations in the previous form.

In [53]:
pprint( eqs_new - Matrix([re(eqs[1,1]),re(eqs[1,0]),im(eqs[1,0])]).subs(ss_comp) )

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦


The steady state solution of this equations is

In [54]:
sol=solve(list(eqs_new),list(rho_vect))
for mu in range(3):
    pprint( {rho_vect[mu]:sol[rho_vect[mu]]} ,num_columns=120)

⎧                2        2            ⎫
⎪              re (Ω) + im (Ω)         ⎪
⎨ρ₂₂: ─────────────────────────────────⎬
⎪        2      2       2          2   ⎪
⎩     4⋅δ  + γ₂₁  + 2⋅re (Ω) + 2⋅im (Ω)⎭
⎧               2⋅δ⋅re(Ω) + γ₂₁⋅im(Ω)      ⎫
⎪re(ρ₂₁): ─────────────────────────────────⎪
⎨            2      2       2          2   ⎬
⎪         4⋅δ  + γ₂₁  + 2⋅re (Ω) + 2⋅im (Ω)⎪
⎩                                          ⎭
⎧               2⋅δ⋅im(Ω) - γ₂₁⋅re(Ω)      ⎫
⎪im(ρ₂₁): ─────────────────────────────────⎪
⎨            2      2       2          2   ⎬
⎪         4⋅δ  + γ₂₁  + 2⋅re (Ω) + 2⋅im (Ω)⎪
⎩                                          ⎭


According to literature [1], the solution should be

In [55]:
s0=2*(re(Omega)**2+im(Omega)**2)/gamma[1,0]**2

s=s0/(1+(2*delta/gamma[1,0])**2)


rho21=-I*Omega/(2*(gamma[1,0]/2-I*delta)*(1+s))

rerho22=( s/(2*(1+s)) ).simplify()
rerho21=re(rho21).simplify()
imrho21=im(rho21).simplify()

test=[ sol[rho[1,1]]-rerho22, sol[re(rho[1,0])]-rerho21, sol[im(rho[1,0])]-imrho21 ]

pprint( [testi.subs({Omega:re(Omega)+I*im(Omega)}).factor() for testi in test] )

[0, 0, 0]


So our development produces the same results as the literature.

The saturation intensity is defined as the intensity needed to accumulate $\frac{1}{4}$ of the population in the excited state when the field is in resonance ($\delta=0$).

In [56]:
saturation_eq=sol[rho[1,1]].subs({delta:0})-1/Integer(4)
pprint( saturation_eq )

       2        2             
     re (Ω) + im (Ω)         1
────────────────────────── - ─
   2       2          2      4
γ₂₁  + 2⋅re (Ω) + 2⋅im (Ω)    


In [57]:
Omega_amp,alpha=symbols("\Omega_a alpha",real=True)
ss={Omega:Omega_amp*cos(alpha)+I*Omega_amp*sin(alpha)}
saturation_eq= saturation_eq.subs(ss).factor().simplify()
pprint(saturation_eq)

           2      2  
  2⋅\Omegaₐ  - γ₂₁   
─────────────────────
  ⎛         2      2⎞
4⋅⎝2⋅\Omegaₐ  + γ₂₁ ⎠


In [58]:
Omega_sat=solve( saturation_eq ,Omega_amp)[1]
pprint(Omega_sat)

√2⋅γ₂₁
──────
  2   


Since $\Omega =e E_0^1 r_{0;21}/\hbar$ it follows that

In [59]:
E0_sat=Omega_sat*hbar/e/r[1][1,0]
pprint(E0_sat)

 √2⋅γ₂₁⋅h̅  
────────────
2⋅e⋅r_{0;21}


The full width at half maximum of $\rho_{22}$ is

In [60]:
hm1,hm2=solve(sol[rho[1,1]]-sol[rho[1,1]].subs({delta:0})/2,delta)
FWHM=hm2-hm1
FWHM=FWHM.subs(ss).simplify()
pprint(FWHM)

   ___________________
  ╱          2      2 
╲╱  2⋅\Omegaₐ  + γ₂₁  


[1]  H.J. Metcalf and P. van der Straten. Laser Cooling and Trapping. Graduate Texts in Contempo-
rary Physics. Springer New York, 2001.